In [1]:
# Ячейка 1: Установка зависимостей
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python pillow matplotlib scikit-learn numpy
!pip install ipywidgets  # для интерактивных элементов

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from sklearn.model_selection import train_test_split
from IPython.display import display, clear_output
import ipywidgets as widgets

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
# Ячейка 2: Создание структуры папок
def create_folder_structure():
    folders = [
        'data/raw/car',
        'data/raw/not_car',
        'data/processed/views/train/front',
        'data/processed/views/train/side', 
        'data/processed/views/train/rear',
        'data/processed/views/val/front',
        'data/processed/views/val/side',
        'data/processed/views/val/rear',
        'data/processed/damage/train/damaged',
        'data/processed/damage/train/intact',
        'data/processed/damage/val/damaged', 
        'data/processed/damage/val/intact',
        'data/processed/dirt/train/dirty',
        'data/processed/dirt/train/clean',
        'data/processed/dirt/val/dirty',
        'data/processed/dirt/val/clean',
        'models',
        'test_images'
    ]
    
    for folder in folders:
        os.makedirs(folder, exist_ok=True)
        print(f"Created: {folder}")
    
    # Создаем README файлы
    for view in ['front', 'side', 'rear']:
        with open(f'data/processed/views/train/{view}/README.txt', 'w') as f:
            f.write(f"Place {view} view car images here")
        with open(f'data/processed/views/val/{view}/README.txt', 'w') as f:
            f.write(f"Place {view} view car images here (validation)")
    
    print("Folder structure created successfully!")

create_folder_structure()

Created: data/raw/car
Created: data/raw/not_car
Created: data/processed/views/train/front
Created: data/processed/views/train/side
Created: data/processed/views/train/rear
Created: data/processed/views/val/front
Created: data/processed/views/val/side
Created: data/processed/views/val/rear
Created: data/processed/damage/train/damaged
Created: data/processed/damage/train/intact
Created: data/processed/damage/val/damaged
Created: data/processed/damage/val/intact
Created: data/processed/dirt/train/dirty
Created: data/processed/dirt/train/clean
Created: data/processed/dirt/val/dirty
Created: data/processed/dirt/val/clean
Created: models
Created: test_images
Folder structure created successfully!


In [3]:
# Ячейка 3: Исправленная функция copy_from_downloads
def copy_from_downloads(target_folder, keywords=None, file_limit=10):
    """
    Копирует изображения из папки Загрузки в целевую папку
    """
    if os.name == 'nt':  # Windows
        downloads_path = os.path.join(os.environ['USERPROFILE'], 'Downloads')
    else:  # Linux/Mac
        downloads_path = os.path.join(os.path.expanduser('~'), 'Downloads')
    
    if not os.path.exists(downloads_path):
        print(f"❌ Папка Загрузки не найдена: {downloads_path}")
        return 0
    
    os.makedirs(target_folder, exist_ok=True)
    
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.JPEG', '*.PNG']
    all_images = []
    
    for ext in image_extensions:
        all_images.extend(glob.glob(os.path.join(downloads_path, ext)))
    
    if keywords:
        filtered_images = []
        for img_path in all_images:
            filename = os.path.basename(img_path).lower()
            # Ищем ЛЮБОЕ из ключевых слов в названии файла
            if any(keyword.lower() in filename for keyword in keywords):
                filtered_images.append(img_path)
        all_images = filtered_images
    
    images_to_copy = all_images[:file_limit]
    
    if not images_to_copy:
        print("❌ Не найдено подходящих изображений в папке Загрузки")
        print(f"Искал файлы содержащие: {keywords}")
        return 0
    
    copied_count = 0
    for img_path in images_to_copy:
        try:
            filename = os.path.basename(img_path)
            target_path = os.path.join(target_folder, filename)
            
            counter = 1
            while os.path.exists(target_path):
                name, ext = os.path.splitext(filename)
                target_path = os.path.join(target_folder, f"{name}_{counter}{ext}")
                counter += 1
            
            shutil.copy2(img_path, target_path)
            print(f"✅ Скопировано: {filename} -> {target_folder}")
            copied_count += 1
            
        except Exception as e:
            print(f"❌ Ошибка при копировании {filename}: {e}")
    
    print(f"📊 Скопировано файлов: {copied_count}/{len(images_to_copy)}")
    return copied_count

# Добавьте эту функцию для отладки
def debug_file_search():
    """Показывает какие файлы находит система"""
    if os.name == 'nt':  # Windows
        downloads_path = os.path.join(os.environ['USERPROFILE'], 'Downloads')
    else:  # Linux/Mac
        downloads_path = os.path.join(os.path.expanduser('~'), 'Downloads')
    
    print(f"🔍 Поиск файлов в: {downloads_path}")
    print("-" * 50)
    
    # Все файлы в загрузках
    all_files = os.listdir(downloads_path)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
    
    print(f"📁 Всего файлов в Загрузках: {len(all_files)}")
    print(f"🖼️ Из них изображений: {len(image_files)}")
    
    if image_files:
        print("📋 Изображения в Загрузках:")
        for img in image_files[:10]:  # первые 10
            print(f"   📄 {img}")
        if len(image_files) > 10:
            print(f"   ... и еще {len(image_files) - 10} изображений")
    
    # Ваши конкретные файлы
    your_files = ['Car', 'car_perfect_sides', 'front', 'rear', 'car_scratch', 'rust_and_scratch', 'car_scratch_and_dent']
    
    print("\n🔎 Поиск ваших файлов:")
    found_count = 0
    for pattern in your_files:
        found_files = [f for f in image_files if pattern.lower() in f.lower()]
        if found_files:
            print(f"   ✅ {pattern}: найдено {len(found_files)} файлов")
            for file in found_files[:3]:  # первые 3
                print(f"      📄 {file}")
            if len(found_files) > 3:
                print(f"      ... и еще {len(found_files) - 3}")
            found_count += len(found_files)
        else:
            print(f"   ❌ {pattern}: не найдено")
    
    print(f"\n📊 Всего найдено ваших файлов: {found_count}")

# Запустите отладку чтобы увидеть что происходит
debug_file_search()

🔍 Поиск файлов в: /Users/ashko/Downloads
--------------------------------------------------
📁 Всего файлов в Загрузках: 56
🖼️ Из них изображений: 21
📋 Изображения в Загрузках:
   📄 IMG_6538.PNG
   📄 IMG_7344.JPG
   📄 IMG_7024.PNG
   📄 6f3f4105-ec19-4569-b106-941b5f325b56.JPG
   📄 _.png
   📄 car_scratch.png
   📄 ChatGPT Image Aug 9, 2025 at 12_26_31 AM.png
   📄 ChatGPT Image Sep 11, 2025 at 12_03_35 AM.png
   📄 IMG_7497.JPG
   📄 IMG_7496.JPG
   ... и еще 11 изображений

🔎 Поиск ваших файлов:
   ✅ Car: найдено 1 файлов
      📄 car_scratch.png
   ❌ car_perfect_sides: не найдено
   ❌ front: не найдено
   ❌ rear: не найдено
   ✅ car_scratch: найдено 1 файлов
      📄 car_scratch.png
   ❌ rust_and_scratch: не найдено
   ❌ car_scratch_and_dent: не найдено

📊 Всего найдено ваших файлов: 2


In [5]:
# Ячейка 3.5: Копирование из ваших папок в нужные директории
def copy_from_your_folders():
    """Копирует файлы из ваших папок в нужные директории обучения"""
    if os.name == 'nt':  # Windows
        downloads_path = os.path.join(os.environ['USERPROFILE'], 'Downloads')
    else:  # Linux/Mac
        downloads_path = os.path.join(os.path.expanduser('~'), 'Downloads')
    
    print(f"🔍 Ищу ваши папки в: {downloads_path}")
    print("-" * 60)
    
    # Ваши папки и куда их копировать
    folder_mapping = {
        'Car': 'data/processed/views/train/front',
        'car_perfect_sides': 'data/processed/views/train/side', 
        'front': 'data/processed/views/train/front',
        'rear': 'data/processed/views/train/rear',
        'car_scratch': 'data/processed/damage/train/damaged',
        'rust_and_scratch': 'data/processed/damage/train/damaged',
        'car_scratch_and_dent': 'data/processed/damage/train/damaged'
    }
    
    total_copied = 0
    found_folders = []
    
    # Ищем каждую папку
    for folder_name, target_folder in folder_mapping.items():
        source_folder = os.path.join(downloads_path, folder_name)
        
        if os.path.exists(source_folder) and os.path.isdir(source_folder):
            found_folders.append(folder_name)
            print(f"✅ Найдена папка: {folder_name}")
            
            # Ищем изображения в папке
            images = []
            for ext in ['*.jpg', '*.jpeg', '*.png']:
                images.extend(glob.glob(os.path.join(source_folder, ext)))
            
            print(f"   📷 Найдено изображений: {len(images)}")
            
            if images:
                # Создаем целевую папку
                os.makedirs(target_folder, exist_ok=True)
                
                # Копируем все изображения
                folder_copied = 0
                for img_path in images:
                    try:
                        filename = os.path.basename(img_path)
                        target_path = os.path.join(target_folder, filename)
                        
                        # Чтобы избежать перезаписи
                        counter = 1
                        while os.path.exists(target_path):
                            name, ext = os.path.splitext(filename)
                            target_path = os.path.join(target_folder, f"{name}_{counter}{ext}")
                            counter += 1
                        
                        shutil.copy2(img_path, target_path)
                        print(f"      ✅ {filename}")
                        folder_copied += 1
                        total_copied += 1
                        
                    except Exception as e:
                        print(f"      ❌ Ошибка с {filename}: {e}")
                
                print(f"   📊 Скопировано из {folder_name}: {folder_copied} файлов")
            else:
                print(f"   ❌ В папке {folder_name} нет изображений")
        else:
            print(f"❌ Папка не найдена: {folder_name}")
    
    print(f"\n📊 Итог:")
    print(f"Найдено папок: {len(found_folders)}")
    print(f"Скопировано файлов: {total_copied}")
    
    if not found_folders:
        print("\n🔎 Содержимое папки Загрузки:")
        all_items = os.listdir(downloads_path)
        folders = [item for item in all_items if os.path.isdir(os.path.join(downloads_path, item))]
        print(f"Всего папок в Загрузках: {len(folders)}")
        for folder in folders:
            print(f"   📁 {folder}")
    
    return total_copied

# Запустите эту функцию
copy_from_your_folders()

# Проверьте результат
check_data_availability()

🔍 Ищу ваши папки в: /Users/ashko/Downloads
------------------------------------------------------------
✅ Найдена папка: Car


NameError: name 'glob' is not defined

In [6]:
# Ячейка 3.5: Проверка и создание тестовых данных
def check_and_create_test_data():
    """Проверяет наличие данных и создает тестовые если нужно"""
    print("🔍 Проверка обучающих данных...")
    
    # Папки которые должны существовать
    required_folders = [
        'data/processed/views/train/front',
        'data/processed/views/train/side',
        'data/processed/views/train/rear',
        'data/processed/views/val/front',
        'data/processed/views/val/side',
        'data/processed/views/val/rear'
    ]
    
    # Проверяем каждую папку
    for folder in required_folders:
        if not os.path.exists(folder):
            os.makedirs(folder, exist_ok=True)
            print(f"✅ Создана папка: {folder}")
    
    # Проверяем есть ли изображения
    train_folders = [
        'data/processed/views/train/front',
        'data/processed/views/train/side',
        'data/processed/views/train/rear'
    ]
    
    has_data = False
    for folder in train_folders:
        if os.path.exists(folder):
            images = [f for f in os.listdir(folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
            if images:
                print(f"📁 {folder}: {len(images)} изображений")
                has_data = True
            else:
                print(f"❌ {folder}: нет изображений")
    
    if not has_data:
        print("\n📝 Создаю тестовые изображения...")
        create_sample_images()
        print("✅ Тестовые изображения созданы!")
    
    return has_data

def create_sample_images():
    """Создает простые тестовые изображения"""
    import numpy as np
    from PIL import Image
    
    # Цвета для разных ракурсов
    colors = {
        'front': (255, 0, 0),    # Красный - перед
        'side': (0, 255, 0),     # Зеленый - бок
        'rear': (0, 0, 255)      # Синий - зад
    }
    
    # Создаем по 5 изображений для каждого ракурса
    for view, color in colors.items():
        folder_path = f'data/processed/views/train/{view}'
        os.makedirs(folder_path, exist_ok=True)
        
        for i in range(5):
            # Создаем простое изображение с цветным прямоугольником
            img_array = np.zeros((100, 100, 3), dtype=np.uint8)
            img_array[20:80, 20:80] = color
            
            img = Image.fromarray(img_array)
            img.save(os.path.join(folder_path, f'{view}_sample_{i+1}.jpg'))
            print(f"✅ Создано: {view}_sample_{i+1}.jpg")
    
    # Создаем немного validation данных
    for view in ['front', 'side', 'rear']:
        val_folder = f'data/processed/views/val/{view}'
        os.makedirs(val_folder, exist_ok=True)
        
        # Копируем по 1 изображению из train в val
        train_folder = f'data/processed/views/train/{view}'
        if os.path.exists(train_folder):
            images = [f for f in os.listdir(train_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
            if images:
                src_path = os.path.join(train_folder, images[0])
                dst_path = os.path.join(val_folder, f'val_{images[0]}')
                shutil.copy2(src_path, dst_path)
                print(f"✅ Создано validation: {dst_path}")

# Проверяем и создаем данные если нужно
has_data = check_and_create_test_data()

# Если данных нет, предлагаем варианты
if not has_data:
    print("\n🎯 Доступные варианты:")
    print("1. Использовать созданные тестовые изображения")
    print("2. Скопировать ваши файлы из Загрузок")
    print("3. Загрузить свои изображения")
    
    # Создаем кнопки для действий
    use_test_btn = widgets.Button(description="✅ Использовать тестовые данные", button_style='success')
    copy_files_btn = widgets.Button(description="📥 Скопировать мои файлы", button_style='info')
    check_again_btn = widgets.Button(description="🔄 Проверить снова", button_style='warning')
    
    test_data_output = widgets.Output()
    
    def on_use_test_click(b):
        with test_data_output:
            clear_output()
            print("🔄 Запускаю обучение на тестовых данных...")
            # Здесь будет вызов функции обучения
    
    def on_copy_files_click(b):
        with test_data_output:
            clear_output()
            print("📥 Используйте вкладку '🎯 Ваши файлы' для копирования")
            auto_sort_your_files()
    
    def on_check_again_click(b):
        with test_data_output:
            clear_output()
            check_and_create_test_data()
    
    use_test_btn.on_click(on_use_test_click)
    copy_files_btn.on_click(on_copy_files_click)
    check_again_btn.on_click(on_check_again_click)
    
    display(widgets.VBox([
        widgets.HTML("<h4>🚨 Нет данных для обучения!</h4>"),
        use_test_btn,
        copy_files_btn, 
        check_again_btn,
        test_data_output
    ]))
else:
    print("✅ Данные для обучения готовы!")

🔍 Проверка обучающих данных...
📁 data/processed/views/train/front: 413 изображений
📁 data/processed/views/train/side: 1037 изображений
📁 data/processed/views/train/rear: 436 изображений
✅ Данные для обучения готовы!


In [8]:
# Ячейка для автоматической сортировки ваших файлов
def auto_sort_your_files():
    """Автоматически сортирует ваши файлы по нужным папкам"""
    found_files = analyze_your_files()
    
    if not found_files:
        print("❌ Файлы не найдены. Проверьте что они в папке Загрузки")
        return
    
    print("\n🚀 Автоматическая сортировка ваших файлов:")
    print("-" * 60)
    
    # Правила сортировки для ваших файлов
    sorting_rules = {
        'front': 'data/processed/views/train/front',
        'rear': 'data/processed/views/train/rear',
        'car_perfect_sides': 'data/processed/views/train/side',
        'car_scratch': 'data/processed/damage/train/damaged',
        'rust_and_scratch': 'data/processed/damage/train/damaged', 
        'car_scratch_and_dent': 'data/processed/damage/train/damaged'
    }
    
    total_copied = 0
    
    for filename, files in found_files.items():
        if filename in sorting_rules:
            target_folder = sorting_rules[filename]
            print(f"\n📁 Сортировка '{filename}' -> {target_folder}:")
            
            for file_path in files:
                try:
                    file_name = os.path.basename(file_path)
                    target_path = os.path.join(target_folder, file_name)
                    
                    # Создаем папку если нет
                    os.makedirs(target_folder, exist_ok=True)
                    
                    # Копируем файл
                    shutil.copy2(file_path, target_path)
                    print(f"   ✅ {file_name}")
                    total_copied += 1
                    
                except Exception as e:
                    print(f"   ❌ Ошибка с {file_name}: {e}")
    
    print(f"\n📊 Итого скопировано: {total_copied} файлов")
    
    # Проверяем результат
    print("\n" + "=" * 50)
    check_data_availability()
    
    # Показываем что получилось
    print("\n🎯 Ваши файлы распределены:")
    for filename, target in sorting_rules.items():
        if filename in found_files:
            files_count = len(found_files[filename])
            print(f"   {filename} -> {target}: {files_count} файлов")

# Создаем кнопку для автоматической сортировки
auto_sort_btn = widgets.Button(description="🚀 Авто-сортировка моих файлов", button_style='success')
auto_sort_output = widgets.Output()

def on_auto_sort_click(b):
    with auto_sort_output:
        clear_output()
        auto_sort_your_files()

auto_sort_btn.on_click(on_auto_sort_click)

print("🎯 Для добавления ваших файлов:")
display(auto_sort_btn)
display(auto_sort_output)

🎯 Для добавления ваших файлов:


Button(button_style='success', description='🚀 Авто-сортировка моих файлов', style=ButtonStyle())

Output()

In [9]:
# Ячейка 4: Классы датасетов
class ViewDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = ['front', 'side', 'rear']
        self.images = []
        
        for class_idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        self.images.append({
                            'path': os.path.join(class_dir, img_name),
                            'label': class_idx
                        })
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_info = self.images[idx]
        image = Image.open(img_info['path']).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, img_info['label']

class DamageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = ['damaged', 'intact']
        self.images = []
        
        for class_idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        self.images.append({
                            'path': os.path.join(class_dir, img_name),
                            'label': class_idx
                        })
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_info = self.images[idx]
        image = Image.open(img_info['path']).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, img_info['label']

# Трансформы
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
# Ячейка 5: Функции обучения
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, model_name):
    """Обучение модели с визуализацией прогресса"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    best_accuracy = 0.0
    
    # Создаем график для live-обновления
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    plt.ion()  # Включаем интерактивный режим
    
    for epoch in range(num_epochs):
        # Обучение
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        
        # Валидация
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader) if len(val_loader) > 0 else 0
        val_accuracy = 100 * val_correct / val_total if val_total > 0 else 0
        
        # Сохраняем метрики
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
        # Сохраняем лучшую модель
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), f'models/{model_name}.pth')
        
        # Обновляем график
        clear_output(wait=True)
        ax1.clear()
        ax2.clear()
        
        ax1.plot(train_losses, label='Train Loss', color='blue')
        ax1.plot(val_losses, label='Val Loss', color='red')
        ax1.set_title('Loss')
        ax1.legend()
        
        ax2.plot(train_accuracies, label='Train Accuracy', color='green')
        ax2.plot(val_accuracies, label='Val Accuracy', color='orange')
        ax2.set_title('Accuracy')
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
        
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
        print(f'Best Val Accuracy: {best_accuracy:.2f}%')
        print('-' * 50)
    
    plt.ioff()
    return model, train_losses, val_losses, train_accuracies, val_accuracies



In [11]:
# Ячейка 6: Обновленная функция обучения
def train_view_classifier():
    """Обучение классификатора ракурсов с проверкой данных"""
    print("🚗 Training View Classifier...")
    
    # Проверяем наличие данных
    if not check_and_create_test_data():
        print("❌ No training data found! Please add images to data folders.")
        return None
    
    # Создаем датасеты
    train_dataset = ViewDataset('data/processed/views/train', train_transform)
    val_dataset = ViewDataset('data/processed/views/val', val_transform)
    
    if len(train_dataset) == 0:
        print("❌ No training images found!")
        print("Please add images to:")
        print("- data/processed/views/train/front/")
        print("- data/processed/views/train/side/") 
        print("- data/processed/views/train/rear/")
        return None
    
    print(f"📊 Training images: {len(train_dataset)}")
    print(f"📊 Validation images: {len(val_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
    
    # Модель
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 3)  # 3 класса: front, side, rear
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Обучение
    model, train_loss, val_loss, train_acc, val_acc = train_model(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=5, model_name='view_classifier'  # Уменьшил до 5 эпох для теста
    )
    
    print("✅ View classifier training completed!")
    return model

# Запускаем обучение
view_model = train_view_classifier()

<Figure size 640x480 with 0 Axes>

Epoch [5/5]
Train Loss: 0.1172, Train Acc: 95.92%
Val Loss: 0.0311, Val Acc: 100.00%
Best Val Accuracy: 100.00%
--------------------------------------------------
✅ View classifier training completed!


In [11]:
# Ячейка 6: Обновленная функция обучения
def train_view_classifier():
    """Обучение классификатора ракурсов с проверкой данных"""
    print("🚗 Training View Classifier...")
    
    # Проверяем наличие данных
    if not check_and_create_test_data():
        print("❌ No training data found! Please add images to data folders.")
        return None
    
    # Создаем датасеты
    train_dataset = ViewDataset('data/processed/views/train', train_transform)
    val_dataset = ViewDataset('data/processed/views/val', val_transform)
    
    if len(train_dataset) == 0:
        print("❌ No training images found!")
        print("Please add images to:")
        print("- data/processed/views/train/front/")
        print("- data/processed/views/train/side/") 
        print("- data/processed/views/train/rear/")
        return None
    
    print(f"📊 Training images: {len(train_dataset)}")
    print(f"📊 Validation images: {len(val_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
    
    # Модель
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 3)  # 3 класса: front, side, rear
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Обучение
    model, train_loss, val_loss, train_acc, val_acc = train_model(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=5, model_name='view_classifier'  # Уменьшил до 5 эпох для теста
    )
    
    print("✅ View classifier training completed!")
    return model

# Запускаем обучение
view_model = train_view_classifier()

<Figure size 640x480 with 0 Axes>

Epoch [5/5]
Train Loss: 0.1172, Train Acc: 95.92%
Val Loss: 0.0311, Val Acc: 100.00%
Best Val Accuracy: 100.00%
--------------------------------------------------
✅ View classifier training completed!


In [11]:
# Ячейка 6: Обновленная функция обучения
def train_view_classifier():
    """Обучение классификатора ракурсов с проверкой данных"""
    print("🚗 Training View Classifier...")
    
    # Проверяем наличие данных
    if not check_and_create_test_data():
        print("❌ No training data found! Please add images to data folders.")
        return None
    
    # Создаем датасеты
    train_dataset = ViewDataset('data/processed/views/train', train_transform)
    val_dataset = ViewDataset('data/processed/views/val', val_transform)
    
    if len(train_dataset) == 0:
        print("❌ No training images found!")
        print("Please add images to:")
        print("- data/processed/views/train/front/")
        print("- data/processed/views/train/side/") 
        print("- data/processed/views/train/rear/")
        return None
    
    print(f"📊 Training images: {len(train_dataset)}")
    print(f"📊 Validation images: {len(val_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
    
    # Модель
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 3)  # 3 класса: front, side, rear
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Обучение
    model, train_loss, val_loss, train_acc, val_acc = train_model(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=5, model_name='view_classifier'  # Уменьшил до 5 эпох для теста
    )
    
    print("✅ View classifier training completed!")
    return model

# Запускаем обучение
view_model = train_view_classifier()

<Figure size 640x480 with 0 Axes>

Epoch [5/5]
Train Loss: 0.1172, Train Acc: 95.92%
Val Loss: 0.0311, Val Acc: 100.00%
Best Val Accuracy: 100.00%
--------------------------------------------------
✅ View classifier training completed!


In [11]:
# Ячейка 6: Обновленная функция обучения
def train_view_classifier():
    """Обучение классификатора ракурсов с проверкой данных"""
    print("🚗 Training View Classifier...")
    
    # Проверяем наличие данных
    if not check_and_create_test_data():
        print("❌ No training data found! Please add images to data folders.")
        return None
    
    # Создаем датасеты
    train_dataset = ViewDataset('data/processed/views/train', train_transform)
    val_dataset = ViewDataset('data/processed/views/val', val_transform)
    
    if len(train_dataset) == 0:
        print("❌ No training images found!")
        print("Please add images to:")
        print("- data/processed/views/train/front/")
        print("- data/processed/views/train/side/") 
        print("- data/processed/views/train/rear/")
        return None
    
    print(f"📊 Training images: {len(train_dataset)}")
    print(f"📊 Validation images: {len(val_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
    
    # Модель
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 3)  # 3 класса: front, side, rear
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Обучение
    model, train_loss, val_loss, train_acc, val_acc = train_model(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=5, model_name='view_classifier'  # Уменьшил до 5 эпох для теста
    )
    
    print("✅ View classifier training completed!")
    return model

# Запускаем обучение
view_model = train_view_classifier()

<Figure size 640x480 with 0 Axes>

Epoch [5/5]
Train Loss: 0.1172, Train Acc: 95.92%
Val Loss: 0.0311, Val Acc: 100.00%
Best Val Accuracy: 100.00%
--------------------------------------------------
✅ View classifier training completed!


In [11]:
# Ячейка 6: Обновленная функция обучения
def train_view_classifier():
    """Обучение классификатора ракурсов с проверкой данных"""
    print("🚗 Training View Classifier...")
    
    # Проверяем наличие данных
    if not check_and_create_test_data():
        print("❌ No training data found! Please add images to data folders.")
        return None
    
    # Создаем датасеты
    train_dataset = ViewDataset('data/processed/views/train', train_transform)
    val_dataset = ViewDataset('data/processed/views/val', val_transform)
    
    if len(train_dataset) == 0:
        print("❌ No training images found!")
        print("Please add images to:")
        print("- data/processed/views/train/front/")
        print("- data/processed/views/train/side/") 
        print("- data/processed/views/train/rear/")
        return None
    
    print(f"📊 Training images: {len(train_dataset)}")
    print(f"📊 Validation images: {len(val_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
    
    # Модель
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 3)  # 3 класса: front, side, rear
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Обучение
    model, train_loss, val_loss, train_acc, val_acc = train_model(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=5, model_name='view_classifier'  # Уменьшил до 5 эпох для теста
    )
    
    print("✅ View classifier training completed!")
    return model

# Запускаем обучение
view_model = train_view_classifier()

<Figure size 640x480 with 0 Axes>

Epoch [5/5]
Train Loss: 0.1172, Train Acc: 95.92%
Val Loss: 0.0311, Val Acc: 100.00%
Best Val Accuracy: 100.00%
--------------------------------------------------
✅ View classifier training completed!


In [12]:
# Ячейка 6.1: Функция для быстрого теста
def quick_test_training():
    """Быстрый тест обучения на минимальных данных"""
    print("⚡ Быстрый тест обучения...")
    
    # Создаем тестовые данные если их нет
    check_and_create_test_data()
    
    # Запускаем обучение
    model = train_view_classifier()
    
    if model is not None:
        print("🎉 Тест обучения завершен успешно!")
        print("Теперь вы можете:")
        print("1. Добавить больше своих изображений")
        print("2. Обучить на полном наборе данных")
        print("3. Протестировать систему")
    else:
        print("❌ Тест не удался. Проверьте данные.")

# Кнопка для быстрого теста
quick_test_btn = widgets.Button(description="⚡ Быстрый тест обучения", button_style='info')
quick_test_output = widgets.Output()

def on_quick_test_click(b):
    with quick_test_output:
        clear_output()
        quick_test_training()

quick_test_btn.on_click(on_quick_test_click)

print("\n⚡ Для быстрого теста:")
display(quick_test_btn)
display(quick_test_output)


⚡ Для быстрого теста:


Button(button_style='info', description='⚡ Быстрый тест обучения', style=ButtonStyle())

Output()

In [13]:
# Ячейка 7: Car Detection System
class CarDetectionSystem:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        
        # Инициализация моделей
        self.car_detector = self._load_car_detector()
        self.view_classifier = self._load_view_classifier()
        self.damage_classifier = self._load_damage_classifier()
        self.dirt_classifier = self._load_dirt_classifier()
        
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def _load_car_detector(self):
        model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        model.eval()
        return model.to(self.device)
    
    def _load_view_classifier(self):
        model = models.resnet18(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, 3)
        # Попробуем загрузить обученные веса
        try:
            model.load_state_dict(torch.load('models/view_classifier.pth', map_location=self.device))
            print("✅ View classifier weights loaded")
        except:
            print("⚠️  Using untrained view classifier")
        return model.to(self.device)
    
    def _load_damage_classifier(self):
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 2)
        return model.to(self.device)
    
    def _load_dirt_classifier(self):
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 2)
        return model.to(self.device)
    
    def detect_car(self, image):
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            predictions = self.car_detector(image_tensor)
        
        car_boxes = []
        if 'boxes' in predictions[0]:
            for box, label, score in zip(predictions[0]['boxes'], 
                                       predictions[0]['labels'], 
                                       predictions[0]['scores']):
                if label == 3 and score > 0.5:  # автомобиль в COCO
                    car_boxes.append(box.cpu().numpy())
        
        return car_boxes
    
    def classify_view(self, car_image):
        image_tensor = self.transform(car_image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            outputs = self.view_classifier(image_tensor)
            _, predicted = torch.max(outputs, 1)
        
        view_classes = ['front', 'side', 'rear']
        return view_classes[predicted.item()]
    
    def analyze_car_state(self, car_image):
        image_tensor = self.transform(car_image).unsqueeze(0).to(self.device)
        
        # Анализ повреждений
        with torch.no_grad():
            damage_output = self.damage_classifier(image_tensor)
            damage_probs = torch.softmax(damage_output, dim=1)
            damage_status = 'damaged' if torch.argmax(damage_probs) == 0 else 'intact'
            damage_confidence = damage_probs[0][torch.argmax(damage_probs)].item()
        
        # Анализ загрязнения
        with torch.no_grad():
            dirt_output = self.dirt_classifier(image_tensor)
            dirt_probs = torch.softmax(dirt_output, dim=1)
            dirt_status = 'dirty' if torch.argmax(dirt_probs) == 0 else 'clean'
            dirt_confidence = dirt_probs[0][torch.argmax(dirt_probs)].item()
        
        return {
            'damage': {'status': damage_status, 'confidence': damage_confidence},
            'dirt': {'status': dirt_status, 'confidence': dirt_confidence}
        }
    
    def process_image(self, image_path):
        if not os.path.exists(image_path):
            return {"error": "Image file not found"}
        
        image = Image.open(image_path).convert('RGB')
        original_image = cv2.imread(image_path)
        original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        
        # Детекция автомобиля
        car_boxes = self.detect_car(image)
        
        if not car_boxes:
            return {"car_detected": False, "message": "No car detected"}
        
        results = []
        for i, box in enumerate(car_boxes):
            # Вырезаем область автомобиля
            x1, y1, x2, y2 = map(int, box)
            car_crop = original_image_rgb[y1:y2, x1:x2]
            car_crop_pil = Image.fromarray(car_crop)
            
            # Классификация ракурса
            view = self.classify_view(car_crop_pil)
            
            # Анализ состояния
            car_state = self.analyze_car_state(car_crop_pil)
            
            results.append({
                'car_id': i,
                'bbox': [x1, y1, x2, y2],
                'view': view,
                'state': car_state
            })
        
        # Визуализация результатов
        self.visualize_results(original_image_rgb, results)
        
        return {
            "car_detected": True,
            "cars_found": len(results),
            "results": results
        }
    
    def visualize_results(self, image, results):
        """Визуализация результатов детекции"""
        plt.figure(figsize=(15, 10))
        img_with_boxes = image.copy()
        
        for result in results:
            x1, y1, x2, y2 = result['bbox']
            view = result['view']
            damage = result['state']['damage']
            dirt = result['state']['dirt']
            
            # Рисуем bounding box
            color = (0, 255, 0) if damage['status'] == 'intact' else (255, 0, 0)
            cv2.rectangle(img_with_boxes, (x1, y1), (x2, y2), color, 3)
            
            # Добавляем текст
            label = f"{view}: {damage['status']}({damage['confidence']:.2f}), {dirt['status']}({dirt['confidence']:.2f})"
            cv2.putText(img_with_boxes, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        plt.imshow(img_with_boxes)
        plt.axis('off')
        plt.title('Car Detection Results')
        plt.show()

# Инициализируем систему
car_system = CarDetectionSystem()

Using device: cpu


/Users/ashko/myenv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/ashko/myenv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ View classifier weights loaded


In [14]:
# Ячейка 8: Исправленный интерактивный интерфейс
def create_interactive_interface():
    """Создает интерактивные кнопки для управления"""
    
    # Кнопка для загрузки изображения
    upload_btn = widgets.FileUpload(description='Upload Image', multiple=False)
    process_btn = widgets.Button(description='Process Image', button_style='success')
    result_output = widgets.Output()
    
    def on_process_btn_clicked(b):
        with result_output:
            clear_output()
            if upload_btn.value:
                try:
                    # Новый способ обработки загруженного файла
                    uploaded_file = upload_btn.value[0]  # Берем первый файл
                    
                    # Сохраняем загруженное изображение
                    with open('test_images/uploaded_image.jpg', 'wb') as f:
                        f.write(uploaded_file['content'])
                    
                    # Обрабатываем изображение
                    print("🔄 Processing image...")
                    results = car_system.process_image('test_images/uploaded_image.jpg')
                    
                    if results.get('car_detected', False):
                        print("✅ Analysis completed!")
                        for car in results['results']:
                            print(f"\n🚗 Car {car['car_id'] + 1}:")
                            print(f"   View: {car['view']}")
                            print(f"   Damage: {car['state']['damage']['status']} "
                                  f"(confidence: {car['state']['damage']['confidence']:.2f})")
                            print(f"   Dirt: {car['state']['dirt']['status']} "
                                  f"(confidence: {car['state']['dirt']['confidence']:.2f})")
                    else:
                        print("❌ No cars detected")
                        
                except Exception as e:
                    print(f"❌ Error processing image: {e}")
                    print("Please try uploading a different image.")
            else:
                print("⚠️ Please upload an image first")
    
    process_btn.on_click(on_process_btn_clicked)
    
    # Отображаем интерфейс
    display(widgets.VBox([
        widgets.HTML("<h2>🚗 Car Analysis System</h2>"),
        widgets.HTML("<p>Upload an image and click 'Process Image' to analyze</p>"),
        upload_btn,
        process_btn,
        result_output
    ]))

# Запускаем интерфейс
create_interactive_interface()

In [15]:
# Ячейка 8.1: Альтернативный интерфейс для загрузки по пути
def create_file_path_interface():
    """Интерфейс для указания пути к файлу"""
    path_text = widgets.Text(
        value='',
        placeholder='Введите путь к изображению (например: test_images/my_car.jpg)',
        description='Path to image:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
    
    process_path_btn = widgets.Button(description='Process from Path', button_style='info')
    path_output = widgets.Output()
    
    def on_process_path_click(b):
        with path_output:
            clear_output()
            image_path = path_text.value.strip()
            
            if not image_path:
                print("⚠️ Please enter a file path")
                return
            
            if not os.path.exists(image_path):
                print(f"❌ File not found: {image_path}")
                print("Available test images:")
                test_images = [f for f in os.listdir('test_images') if f.endswith(('.jpg', '.jpeg', '.png'))]
                for img in test_images:
                    print(f"   test_images/{img}")
                return
            
            print(f"🔄 Processing: {image_path}")
            results = car_system.process_image(image_path)
            
            if results.get('car_detected', False):
                print("✅ Analysis completed!")
                for car in results['results']:
                    print(f"\n🚗 Car {car['car_id'] + 1}:")
                    print(f"   View: {car['view']}")
                    print(f"   Damage: {car['state']['damage']['status']} "
                          f"(confidence: {car['state']['damage']['confidence']:.2f})")
                    print(f"   Dirt: {car['state']['dirt']['status']} "
                          f"(confidence: {car['state']['dirt']['confidence']:.2f})")
            else:
                print("❌ No cars detected")
    
    process_path_btn.on_click(on_process_path_click)
    
    return widgets.VBox([
        widgets.HTML("<h3>📁 Или укажите путь к файлу:</h3>"),
        path_text,
        process_path_btn,
        path_output
    ])

# Добавляем оба интерфейса
print("📤 Выберите способ загрузки изображения:")
display(widgets.VBox([
    widgets.HTML("<h2>🚗 Car Analysis System</h2>"),
    widgets.HTML("<h3>Способ 1: Загрузка файла</h3>"),
    create_interactive_interface(),
    widgets.HTML("<h3>Способ 2: Указание пути</h3>"),
    create_file_path_interface()
]))

📤 Выберите способ загрузки изображения:


TraitError: The 'children' trait of a VBox instance contains an Instance of a TypedTuple which expected a Widget, not the NoneType None.

In [ ]:
# Ячейка 8.2: Просмотр тестовых изображений
def show_test_images():
    """Показывает доступные тестовые изображения"""
    test_dir = 'test_images'
    if not os.path.exists(test_dir):
        os.makedirs(test_dir, exist_ok=True)
        print(f"📁 Created test_images folder")
        return
    
    images = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    if not images:
        print("❌ No test images found in 'test_images/' folder")
        print("Please add some images or use the uploader")
        return
    
    print("📸 Available test images:")
    for i, img in enumerate(images):
        print(f"   {i+1}. {img}")
    
    # Показываем превью первых 3 изображений
    print("\n🎨 Preview (first 3 images):")
    fig, axes = plt.subplots(1, min(3, len(images)), figsize=(15, 5))
    
    if min(3, len(images)) == 1:
        axes = [axes]
    
    for i, img_name in enumerate(images[:3]):
        img_path = os.path.join(test_dir, img_name)
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].set_title(img_name)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Кнопка для показа тестовых изображений
show_test_btn = widgets.Button(description="👀 Показать тестовые изображения", button_style='info')
test_images_output = widgets.Output()

def on_show_test_click(b):
    with test_images_output:
        clear_output()
        show_test_images()

show_test_btn.on_click(on_show_test_click)

print("\n📸 Доступные изображения:")
display(show_test_btn)
display(test_images_output)

In [ ]:
# Ячейка 9: Тестовые функции
def test_with_sample_images():
    """Тестирование системы на нескольких примерах"""
    test_images = [f for f in os.listdir('test_images') if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    if not test_images:
        print("No test images found in 'test_images/' folder")
        return
    
    for img_name in test_images[:3]:  # тестируем первые 3 изображения
        img_path = os.path.join('test_images', img_name)
        print(f"\n🔍 Testing: {img_name}")
        print("=" * 50)
        
        results = car_system.process_image(img_path)
        
        if results.get('car_detected', False):
            for car in results['results']:
                print(f"🚗 Car: {car['view']} view")
                print(f"   Damage: {car['state']['damage']['status']} "
                      f"(confidence: {car['state']['damage']['confidence']:.2f})")
                print(f"   Dirt: {car['state']['dirt']['status']} "
                      f"(confidence: {car['state']['dirt']['confidence']:.2f})")
        else:
            print("❌ No cars detected")
        print("=" * 50)

# Запускаем тестирование
test_with_sample_images()

In [ ]:
# Ячейка 10: Утилиты для работы с моделями
def save_all_models():
    """Сохраняет все модели"""
    os.makedirs('models', exist_ok=True)
    
    # Сохраняем view classifier если он обучен
    if hasattr(car_system, 'view_classifier'):
        torch.save(car_system.view_classifier.state_dict(), 'models/view_classifier.pth')
        print("✅ View classifier saved")
    
    print("All models saved in 'models/' folder")

def load_all_models():
    """Загружает все сохраненные модели"""
    if os.path.exists('models/view_classifier.pth'):
        car_system.view_classifier.load_state_dict(
            torch.load('models/view_classifier.pth', map_location=car_system.device)
        )
        print("✅ View classifier loaded")
    
    print("All available models loaded")

# Сохраняем модели
save_all_models()

# Создаем кнопку для перезагрузки моделей
reload_btn = widgets.Button(description='Reload Models', button_style='info')
reload_output = widgets.Output()

def on_reload_clicked(b):
    with reload_output:
        clear_output()
        load_all_models()
        print("🔄 Models reloaded!")

reload_btn.on_click(on_reload_clicked)
display(reload_btn)
display(reload_output)